In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics, linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor

data_path = 'data/'

#lgb25 = pd.read_csv('../lgCV_2505.csv.gz', compression='gzip')
lgb27 = pd.read_csv('../lgCV_2705B.csv.gz', compression='gzip')
lgb31 = pd.read_csv('../lgCV_3105.csv.gz', compression='gzip')
rnn = pd.read_csv('../rnnCV_2805.csv.gz', compression='gzip')
mlp = pd.read_csv('../mlpCV_2505.csv.gz', compression='gzip')
truth = pd.read_csv('../../%strain.csv.zip'%(data_path), compression='zip')
y = truth['deal_probability'].values
truth.drop('deal_probability', 1)
test =  pd.read_csv('../../%stest.csv.zip'%(data_path), compression='zip')
test['deal_probability']=float('NAN') 
truth = pd.concat([truth,test[truth.columns]],axis=0)

In [2]:
#lgb25.rename(columns={'deal_probability': 'lgb25_preds' }, inplace=True)
lgb31.rename(columns={'deal_probability': 'lgb31_preds' }, inplace=True)
lgb27.rename(columns={'deal_probability': 'lgb27_preds' }, inplace=True)
mlp.rename(columns={'deal_probability': 'mlp_preds' }, inplace=True)
preds_df = lgb27.merge(rnn, on='item_id')\
                .merge(mlp, on='item_id')\
                .merge(lgb31, on='item_id')\
                .merge(truth, on='item_id',how='left')

In [3]:
preds_df['difference_l27_r']=preds_df['lgb27_preds'] - preds_df['rnn_preds']
preds_df['difference_l27_m']=preds_df['lgb27_preds'] - preds_df['mlp_preds']
#preds_df['difference_l27_l25']=preds_df['lgb27_preds'] - preds_df['lgb25_preds']
preds_df['price'].fillna(-1,inplace=True)

In [4]:
idx = preds_df['deal_probability']==preds_df['deal_probability']
print(idx.value_counts())
for col in [c for c in preds_df.columns if '_preds' in c]:
    print('RMSE %s: '%(col), np.sqrt(metrics.mean_squared_error(preds_df['deal_probability'][idx].values, preds_df[col][idx].values)))

True     1503424
False     508438
Name: deal_probability, dtype: int64
RMSE lgb27_preds:  0.21681527582821059
RMSE rnn_preds:  0.21771476573780568
RMSE mlp_preds:  0.2187544351068867
RMSE lgb31_preds:  0.21681135578306754


In [6]:
preds_df[[c for c in preds_df.columns if '_preds' in c]].corr()

,lgb27_preds,rnn_preds,mlp_preds,lgb31_preds
lgb27_preds,1.000000,0.919586,0.920325,0.990926
rnn_preds,0.919586,1.000000,0.932392,0.914960
mlp_preds,0.920325,0.932392,1.000000,0.916215
lgb31_preds,0.990926,0.914960,0.916215,1.000000


In [13]:
cols = ['lgb27_preds', 'lgb31_preds', 'rnn_preds','mlp_preds','price', \
        'difference_l27_r', 'difference_l27_m', \
        'region', 'param_1', 'parent_category_name', 'category_name' ]
for col in ['region', 'param_1', 'parent_category_name', 'category_name']:
    preds_df[col] = LabelEncoder().fit_transform(preds_df[col].fillna("0"))
train_df = preds_df[~preds_df['deal_probability'].isnull()]
test_df = preds_df[preds_df['deal_probability'].isnull()]

In [16]:
n_estimators = 4000
train_X, valid_X, train_y, valid_y = train_test_split(train_df[cols], y, train_size=.8, random_state=12345)
eval_set = [(train_X,train_y),(valid_X,valid_y)]

/home/darragh/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [17]:
len(train_X)

1202739

In [21]:
%%time
clf = LGBMRegressor(n_estimators=n_estimators, 
                    max_depth=-1, 
                    feature_fraction= 0.5,
                    num_leaves=32, 
                    learning_rate=.01)#, device='gpu')
clf.fit(train_X, train_y, early_stopping_rounds=80, 
        eval_set=eval_set, eval_metric='rmse', verbose=100, 
        categorical_feature=['param_1','region','parent_category_name','category_name'])

/home/darragh/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is ['category_name', 'param_1', 'parent_category_name', 'region']
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 80 rounds.
[100]	training's rmse: 0.221553	valid_1's rmse: 0.221868
[200]	training's rmse: 0.215097	valid_1's rmse: 0.215614
[300]	training's rmse: 0.21393	valid_1's rmse: 0.214611
[400]	training's rmse: 0.213603	valid_1's rmse: 0.214419
[500]	training's rmse: 0.213425	valid_1's rmse: 0.214369
[600]	training's rmse: 0.213301	valid_1's rmse: 0.214346
[700]	training's rmse: 0.213194	valid_1's rmse: 0.214323
[800]	training's rmse: 0.213092	valid_1's rmse: 0.214309
[900]	training's rmse: 0.212998	valid_1's rmse: 0.214298
[1000]	training's rmse: 0.21292	valid_1's rmse: 0.21429
[1100]	training's rmse: 0.212836	valid_1's rmse: 0.214282
[1200]	training's rmse: 0.21276	valid_1's rmse: 0.214277
[1300]	training's rmse: 0.212686	valid_1's rmse: 0.214273
[1400]	training's rmse: 0.212609	valid_1's rmse: 0.21427
[1500]	training's rmse: 0.212542	valid_1's rmse: 0.214267
[1600]	training's rmse: 0.212469	valid_1's rmse: 0.214262
[1700]	training's rmse: 

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       feature_fraction=0.5, learning_rate=0.01, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=4000, n_jobs=-1, num_leaves=32, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=1)

In [22]:
n_estimators = 2868
train_X = train_df[cols]
train_y = y
eval_set = [(train_X,train_y)]
len(train_X)

1503424

In [25]:
%%time
clf = LGBMRegressor(n_estimators=n_estimators, 
                    max_depth=-1, 
                    feature_fraction= 0.5,
                    num_leaves=32, 
                    learning_rate=.01)#, device='gpu')

clf.fit(train_X, train_y, early_stopping_rounds=80, 
        eval_set=eval_set, eval_metric='rmse', verbose=100, 
        categorical_feature=['param_1','region','parent_category_name','category_name'])

/home/darragh/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is ['category_name', 'param_1', 'parent_category_name', 'region']
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 80 rounds.
[100]	training's rmse: 0.221618
[200]	training's rmse: 0.215198
[300]	training's rmse: 0.214056
[400]	training's rmse: 0.213748
[500]	training's rmse: 0.213587
[600]	training's rmse: 0.213475
[700]	training's rmse: 0.213382
[800]	training's rmse: 0.213295
[900]	training's rmse: 0.213223
[1000]	training's rmse: 0.213156
[1100]	training's rmse: 0.213087
[1200]	training's rmse: 0.213022
[1300]	training's rmse: 0.212962
[1400]	training's rmse: 0.212903
[1500]	training's rmse: 0.212843
[1600]	training's rmse: 0.212781
[1700]	training's rmse: 0.21272
[1800]	training's rmse: 0.212664
[1900]	training's rmse: 0.21261
[2000]	training's rmse: 0.212557
[2100]	training's rmse: 0.212505
[2200]	training's rmse: 0.212448
[2300]	training's rmse: 0.212396
[2400]	training's rmse: 0.212339
[2500]	training's rmse: 0.212286
[2600]	training's rmse: 0.212235
[2700]	training's rmse: 0.212186
[2800]	training's rmse: 0.21213
Did not meet early stoppin

In [26]:
test_df['deal_probability'] = clf.predict(test_df[cols])
test_df['deal_probability'] = np.clip(test_df['deal_probability'], .0001, .9999)
test_df[['item_id', 'deal_probability']].to_csv('../lgbbsub_0106L2.csv.gz', compression='gzip', index=False, header=True)

/home/darragh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/darragh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [27]:
test_df[['item_id', 'deal_probability']].head()

,item_id,deal_probability
1503424,6544e41a8817,0.411257
1503425,65b9484d670f,0.166102
1503426,8bab230b2ecd,0.140598
1503427,8e348601fefc,0.076480
1503428,8bd2fe400b89,0.224623
